In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Replace 'filename.csv' with the actual CSV file name in the dataset
train = '/kaggle/input/f1nalyze-datathon-ieeecsmuj/train.csv'
train_ = pd.read_csv(train)

# Display the first few rows of the DataFrame
train_.head()


In [ ]:
train_.shape

In [ ]:
test = '/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv'
test_ = pd.read_csv(test)

# Display the first few rows of the DataFrame
test_.head()


In [ ]:
test_.shape

In [ ]:
validation = '/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv'
validation_ = pd.read_csv(validation)

# Display the first few rows of the DataFrame
validation_.head()

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the datasets
train = pd.read_csv('/kaggle/input/f1nalyze-datathon-ieeecsmuj/train.csv')
test = pd.read_csv('/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv')
validation = pd.read_csv('/kaggle/input/f1nalyze-datathon-ieeecsmuj/validation.csv')
train = train[:1000]
test = test[:1000]
validation = validation[:1000]

# Display the first few rows of each DataFrame
print("Train Data:")
print(train.head())

print("\nTest Data:")
print(test.head())

print("\nValidation Data:")
print(validation.head())

# Check for missing values
print("\nMissing values in Train Data:")
print(train.isnull().sum())

print("\nMissing values in Test Data:")
print(test.isnull().sum())

print("\nMissing values in Validation Data:")
print(validation.isnull().sum())

# Convert non-numeric values to NaN and handle them
train.replace('\\N', np.nan, inplace=True)
test.replace('\\N', np.nan, inplace=True)
validation.replace('\\N', np.nan, inplace=True)

# Convert columns to appropriate data types
train = train.apply(pd.to_numeric, errors='ignore')
test = test.apply(pd.to_numeric, errors='ignore')
validation = validation.apply(pd.to_numeric, errors='ignore')

# Fill missing values (can also use other strategies like mean/median imputation)
train.fillna(method='ffill', inplace=True)
test.fillna(method='ffill', inplace=True)
validation.fillna(method='ffill', inplace=True)


In [ ]:
# Visualize the distribution of the target variable (positionOrder) in the train set
sns.histplot(train['positionOrder'], bins=20, kde=True)
plt.title('Distribution of Finishing Positions')
plt.xlabel('Finishing Position')
plt.ylabel('Frequency')
plt.show()

# Visualize the correlation matrix for numeric columns only
numeric_cols = train.select_dtypes(include=[np.number]).columns
corr_matrix = train[numeric_cols].corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Feature Engineering: Creating new features from existing columns
train['grid_position_diff'] = train['grid'] - train['positionOrder']
test['grid_position_diff'] = test['grid'] - test['positionOrder']
validation['grid_position_diff'] = validation['grid'] - validation['positionOrder']

# Select relevant features for modeling
features = ['racerId', 'driverId', 'constructorId', 'grid', 'points', 'grid_position_diff']
target = 'positionOrder'

X_train = train[features]
y_train = train[target]

X_test = test[features]
X_val = validation[features]
y_val = validation[target]


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialize a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train_split, y_train_split)

# Get the best model
best_rf = grid_search.best_estimator_

# Evaluate the model
y_train_pred = best_rf.predict(X_train_split)
y_val_pred = best_rf.predict(X_val_split)

train_rmse = np.sqrt(mean_squared_error(y_train_split, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val_split, y_val_pred))

print(f"Training RMSE: {train_rmse}")
print(f"Validation RMSE: {val_rmse}")

# Test set predictions
y_test_pred = best_rf.predict(X_test)


In [ ]:
# Evaluate the model on the validation set
val_rmse = np.sqrt(mean_squared_error(y_val, y_test_pred))
print(f"Test RMSE: {val_rmse}")

# Save the predictions for submission
submission = pd.DataFrame({'resultId': test['resultId'], 'predicted_positionOrder': y_test_pred})
submission.to_csv('submission.csv', index=False)
